# Convergence speed analysis

This notebook assumes the user is in his own machine.

**Change this if necessary:**

In [58]:
USER = "st76i1_1"
GROUP = "st76i1"
PASSWORD = input()

### Generate runs

In [ ]:
# make upload
!sshpass -p {PASSWORD} \
    scp -r ../../optimizer/ {USER}@chome.metz.supelec.fr:/usr/users/{GROUP}/{USER}/


In [92]:
# run algorithm on cluster
runs = [
    "--algorithm hill_climbing --steps 100",
    "--algorithm greedy --steps 10",
    "--algorithm tabu_greedy --steps 10 --hparams '{\"n_tabu\":5}'",
    "--algorithm simulated_annealing --steps 100 --hparams '{\"t0\":20}'",
    "--algorithm csa --steps 100",
    "--algorithm cmaes --steps 100",
]

import pexpect

child = pexpect.spawn(f"sshpass -p {PASSWORD} \
                      ssh -tt {USER}@chome.metz.supelec.fr \
                      srun -p cpu_tp -N 1 -n 32 --pty bash",
                      timeout=5*60)

prompt = f"{USER}@kyle[0-9]+:"

for i in range(len(runs)):
    occupied = child.expect([prompt, "srun: job [1-9]+ queued and waiting for resources"])

    if occupied == 0:
        cmd = f"python3 -m optimizer.main {runs[i]} --log convergence_run_{i}.log"
        child.sendline(cmd)
        running = 1
        while running == 1:
            running = child.expect(['Run finished', '\n'])
            output = child.before.decode()
            print(output)
        output = child.after.decode()
        print(output)

    elif occupied == 1:
        raise Exception("Error: no resource available. Try running `mysrun` and `scancel` in the remote machine.")
    
child.close()


 ~st76i1_1@kyle68:~$ python3 -m optimizer.main --algorithm hill_climbing --steps 100 --log convergence_run_0.log
MPI startup(): PMI server not found. Please set I_MPI_PMI_LIBRARY variable if it is not a singleton case.
[info] [Me=0] Args:
	algorithm: hill_climbing
	batch: False
	hparams: {}
	log: convergence_run_0.log
	phase: deploy
	problem_size: [256, 256, 256]
	seed: 33
	steps: 100
[info] [Me=0] Hyperparameters:
[info] [Me=0] real seed: 33
[info] [Me=0] Executed command: /usr/bin/mpirun -np 1 -map-by ppr:1:node:PE=16 python3 -m optimizer.main --algorithm hill_climbing --hparams '{}' --log convergence_run_0.log --phase run --problem_size 256 256 256 --seed 33 --steps 100 
Unable to init server: Could not connect: Connection refused
Unable to init server: Impossible de se connecter : Connection refused

(main.py:629400): Gdk-CRITICAL **: 01:45:33.789: gdk_cursor_new_for_display: assertion 'GDK_IS_DISPLAY (display)' failed
[info] [Me=0] Args:
	algorithm: hill_climbing
	batch: False
	h

KeyboardInterrupt: 

In [88]:
# make get_logs
!mkdir -p tmp/
!sshpass -p {PASSWORD} \
    scp '{USER}@chome.metz.supelec.fr:/usr/users/{GROUP}/{USER}/convergence_*.log' ./tmp/